# ISCA-k Diagnostic Tests

Este notebook contém testes diagnósticos controlados para verificar o comportamento interno do ISCA-k.

**Objectivo**: Criar cenários sintéticos onde sabemos exactamente o que deveria acontecer e verificar se o algoritmo se comporta como esperado.

In [ ]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

# Imports do ISCA-k
from iscak_core import ISCAkCore
from core.adaptive_k import adaptive_k_hybrid
from core.mi_calculator import calculate_mi_mixed
from core.distances import (
    weighted_euclidean_batch, 
    mixed_distance_pds,
    range_normalized_mixed_distance
)
from preprocessing.type_detection import MixedTypeHandler
from preprocessing.scaling import get_scaled_data, compute_range_factors

np.random.seed(42)
print("Imports OK")

---
## Funções Auxiliares de Diagnóstico

Funções para ver "debaixo do capot" o que o ISCA-k está a fazer.

In [ ]:
def diagnose_imputation(imputer, data, target_col, row_idx, true_value=None):
    """
    Diagnóstico detalhado de uma imputação específica.
    
    Mostra: pesos MI, candidatos a donor, distâncias, k escolhido, vizinhos finais.
    """
    print(f"\n{'='*70}")
    print(f"DIAGNÓSTICO: Linha {row_idx}, Coluna '{target_col}'")
    print(f"{'='*70}")
    
    # Dados da linha a imputar
    row_data = data.loc[row_idx]
    print(f"\nDados da linha {row_idx}:")
    print(row_data.to_string())
    
    if true_value is not None:
        print(f"\nValor REAL (ground truth): {true_value}")
    
    # Features disponíveis (não-missing)
    feature_cols = [c for c in data.columns if c != target_col]
    avail_features = [c for c in feature_cols if not pd.isna(row_data[c])]
    missing_features = [c for c in feature_cols if pd.isna(row_data[c])]
    
    print(f"\nFeatures disponíveis ({len(avail_features)}): {avail_features}")
    print(f"Features em falta ({len(missing_features)}): {missing_features}")
    
    # Pesos MI
    if imputer.mi_matrix is not None:
        print(f"\n--- PESOS MI para prever '{target_col}' ---")
        mi_scores = imputer.mi_matrix.loc[feature_cols, target_col]
        weights = mi_scores.values / mi_scores.sum() if mi_scores.sum() > 0 else np.ones(len(mi_scores)) / len(mi_scores)
        
        for col, mi, w in sorted(zip(feature_cols, mi_scores.values, weights), key=lambda x: -x[2]):
            avail_marker = "✓" if col in avail_features else "✗"
            print(f"  {avail_marker} {col}: MI={mi:.4f}, peso={w:.4f}")
    
    # Donors potenciais
    complete_mask = ~data[target_col].isna()
    donors = data[complete_mask]
    print(f"\n--- DONORS POTENCIAIS ({len(donors)} linhas com target preenchido) ---")
    
    # Calcular distâncias manualmente para diagnóstico
    scaled_data = imputer._get_scaled_data(data)
    range_factors = imputer._compute_range_factors(data, scaled_data)
    
    print(f"\n--- RANGE FACTORS ---")
    for i, col in enumerate(data.columns):
        if col != target_col:
            print(f"  {col}: {range_factors[i]:.4f}")
    
    # Calcular distância para cada donor
    distances_info = []
    sample_scaled = scaled_data.loc[row_idx, feature_cols].values.astype(np.float64)
    
    for donor_idx in donors.index:
        donor_scaled = scaled_data.loc[donor_idx, feature_cols].values.astype(np.float64)
        donor_target = donors.loc[donor_idx, target_col]
        
        # Contar overlap
        overlap = sum(1 for s, d in zip(sample_scaled, donor_scaled) if not (np.isnan(s) or np.isnan(d)))
        
        # Calcular distância simples (euclidiana ponderada)
        dist = 0.0
        n_used = 0
        for j, (s, d, w) in enumerate(zip(sample_scaled, donor_scaled, weights)):
            if not (np.isnan(s) or np.isnan(d)):
                diff = abs(s - d)
                dist += (diff * w) ** 2
                n_used += 1
        dist = np.sqrt(dist) if n_used > 0 else np.inf
        
        distances_info.append({
            'donor_idx': donor_idx,
            'distance': dist,
            'overlap': overlap,
            'target': donor_target
        })
    
    # Ordenar por distância
    distances_info.sort(key=lambda x: x['distance'])
    
    print(f"\n--- TOP 10 DONORS (por distância) ---")
    for i, info in enumerate(distances_info[:10]):
        print(f"  {i+1}. Linha {info['donor_idx']}: dist={info['distance']:.4f}, overlap={info['overlap']}/{len(feature_cols)}, target={info['target']:.2f}")
    
    # Simular escolha de k
    distances_arr = np.array([d['distance'] for d in distances_info])
    targets_arr = np.array([d['target'] for d in distances_info])
    
    # Verificar se é categórico
    is_cat = (target_col in imputer.mixed_handler.nominal_cols or 
              target_col in imputer.mixed_handler.binary_cols)
    
    k = adaptive_k_hybrid(
        distances_arr, targets_arr,
        min_k=imputer.min_friends, max_k=imputer.max_friends,
        alpha=imputer.adaptive_k_alpha, is_categorical=is_cat
    )
    
    # Calcular trust components
    k_eval = min(imputer.max_friends, len(distances_arr))
    closest_dist = distances_arr[:k_eval]
    closest_vals = targets_arr[:k_eval]
    
    mean_dist = np.mean(closest_dist[np.isfinite(closest_dist)])
    density_trust = 1.0 / (1.0 + mean_dist) if np.isfinite(mean_dist) else 0.5
    
    if is_cat:
        unique, counts = np.unique(closest_vals, return_counts=True)
        consistency_trust = counts.max() / len(closest_vals) if len(counts) > 0 else 0.5
    else:
        mean_val = np.mean(closest_vals)
        std_val = np.std(closest_vals)
        cv = std_val / abs(mean_val) if abs(mean_val) > 1e-10 else std_val
        consistency_trust = 1.0 / (1.0 + cv)
    
    print(f"\n--- ADAPTIVE K ---")
    print(f"  density_trust: {density_trust:.4f} (mean_dist={mean_dist:.4f})")
    print(f"  consistency_trust: {consistency_trust:.4f}")
    print(f"  k escolhido: {k} (range [{imputer.min_friends}, {imputer.max_friends}])")
    
    # Vizinhos finais
    final_neighbors = distances_info[:k]
    print(f"\n--- VIZINHOS FINAIS (k={k}) ---")
    for i, info in enumerate(final_neighbors):
        print(f"  {i+1}. Linha {info['donor_idx']}: dist={info['distance']:.4f}, target={info['target']:.2f}")
    
    # Calcular valor imputado
    neighbor_dists = np.array([d['distance'] for d in final_neighbors])
    neighbor_vals = np.array([d['target'] for d in final_neighbors])
    
    if is_cat:
        # Votação ponderada
        weighted_votes = {}
        for val, dist in zip(neighbor_vals, neighbor_dists):
            w = 1 / (dist + 1e-6)
            weighted_votes[val] = weighted_votes.get(val, 0) + w
        imputed = max(weighted_votes.items(), key=lambda x: x[1])[0]
    else:
        # Média ponderada por distância inversa
        if np.any(neighbor_dists < 1e-10):
            imputed = np.mean(neighbor_vals[neighbor_dists < 1e-10])
        else:
            w = 1 / (neighbor_dists + 1e-6)
            w = w / w.sum()
            imputed = np.average(neighbor_vals, weights=w)
            print(f"\n  Pesos IDW: {w}")
    
    print(f"\n--- RESULTADO ---")
    print(f"  Valor imputado: {imputed:.4f}")
    if true_value is not None:
        error = abs(imputed - true_value)
        print(f"  Valor real: {true_value:.4f}")
        print(f"  Erro absoluto: {error:.4f}")
        print(f"  Status: {'✓ BOM' if error < 1.0 else '✗ MAU'}")
    
    return imputed

In [ ]:
def run_test_with_diagnostics(data_complete, missing_indices, target_col, test_name, use_pds=True):
    """
    Executa um teste completo com diagnósticos.
    
    Args:
        data_complete: DataFrame completo (ground truth)
        missing_indices: Lista de (row_idx, col_name) para tornar missing
        target_col: Coluna target principal para diagnóstico detalhado
        test_name: Nome do teste
        use_pds: Se usar PDS ou não
    """
    print(f"\n{'#'*70}")
    print(f"# TESTE: {test_name}")
    print(f"{'#'*70}")
    
    # Criar cópia com missings
    data_missing = data_complete.copy()
    true_values = {}
    
    for row_idx, col in missing_indices:
        true_values[(row_idx, col)] = data_complete.loc[row_idx, col]
        data_missing.loc[row_idx, col] = np.nan
    
    print(f"\nDataset: {data_missing.shape[0]} linhas x {data_missing.shape[1]} colunas")
    print(f"Missings introduzidos: {len(missing_indices)}")
    print(f"PDS: {use_pds}")
    
    print(f"\n--- DADOS COMPLETOS (ground truth) ---")
    print(data_complete.to_string())
    
    print(f"\n--- DADOS COM MISSINGS ---")
    print(data_missing.to_string())
    
    # Executar ISCA-k
    imputer = ISCAkCore(verbose=True, use_pds=use_pds, min_friends=2, max_friends=5)
    result = imputer.impute(data_missing, interactive=False)
    
    print(f"\n--- RESULTADO IMPUTAÇÃO ---")
    print(result.to_string())
    
    # Verificar erros
    print(f"\n--- VERIFICAÇÃO DE ERROS ---")
    total_error = 0
    n_errors = 0
    
    for (row_idx, col), true_val in true_values.items():
        imputed_val = result.loc[row_idx, col]
        error = abs(imputed_val - true_val)
        total_error += error
        n_errors += 1
        
        status = "✓" if error < 1.0 else "✗"
        print(f"  {status} Linha {row_idx}, Col '{col}': imputado={imputed_val:.2f}, real={true_val:.2f}, erro={error:.2f}")
    
    mae = total_error / n_errors if n_errors > 0 else 0
    print(f"\nMAE total: {mae:.4f}")
    
    # Diagnóstico detalhado para o primeiro missing
    if missing_indices:
        first_row, first_col = missing_indices[0]
        print(f"\n{'='*70}")
        print(f"DIAGNÓSTICO DETALHADO para linha {first_row}, coluna '{first_col}'")
        diagnose_imputation(imputer, data_missing, first_col, first_row, true_values[(first_row, first_col)])
    
    # Stats da execução
    print(f"\n--- ESTATÍSTICAS DE EXECUÇÃO ---")
    stats = imputer.execution_stats
    print(f"  Fase 2 activada: {stats.get('phase2_activated', 'N/A')}")
    print(f"  Fase 2 ciclos: {stats.get('phase2_cycles', 'N/A')}")
    print(f"  Fase 2 imputados: {stats.get('phase2_imputed', 'N/A')}")
    
    return result, imputer, mae

---
## Teste 1: Vizinhos Óbvios (Sanity Check)

**Objectivo**: Verificar se o algoritmo escolhe os vizinhos correctos num cenário simples.

Criamos dois clusters bem separados e verificamos se a imputação usa vizinhos do cluster correcto.

### Teste 1.1: Dataset pequeno (7 linhas, 3 features)

In [ ]:
# Dataset com dois clusters óbvios
data_1_1 = pd.DataFrame({
    'A': [1.0, 1.1, 1.2, 5.0, 5.1, 5.2, 1.05],
    'B': [1.0, 1.1, 1.2, 5.0, 5.1, 5.2, 1.05],
    'C': [1.0, 1.1, 1.2, 5.0, 5.1, 5.2, 1.05],
    'Target': [10.0, 11.0, 12.0, 50.0, 51.0, 52.0, 10.5]  # Linha 6 será missing
})

# Comportamento esperado:
# - Vizinhos de linha 6 devem ser {0, 1, 2} (não {3, 4, 5})
# - Valor imputado deve ser ~10.5 (não ~50)

result, imputer, mae = run_test_with_diagnostics(
    data_1_1, 
    missing_indices=[(6, 'Target')],
    target_col='Target',
    test_name="1.1 - Vizinhos Óbvios (7x4, 1 missing)"
)

# Verificação
imputed_val = result.loc[6, 'Target']
expected_range = (9.5, 12.5)  # Deve estar próximo de 10-12
assert expected_range[0] <= imputed_val <= expected_range[1], f"FALHOU: {imputed_val} não está em {expected_range}"
print(f"\n✓ TESTE 1.1 PASSOU: valor imputado {imputed_val:.2f} está no range esperado {expected_range}")

### Teste 1.2: Dataset médio (20 linhas, 5 features, 10% missing)

In [ ]:
# Dois clusters com mais linhas
np.random.seed(42)

# Cluster 1: valores ~1, target ~10
cluster1 = pd.DataFrame({
    'A': np.random.normal(1.0, 0.1, 10),
    'B': np.random.normal(1.0, 0.1, 10),
    'C': np.random.normal(1.0, 0.1, 10),
    'D': np.random.normal(1.0, 0.1, 10),
    'Target': np.random.normal(10.0, 0.5, 10)
})

# Cluster 2: valores ~5, target ~50
cluster2 = pd.DataFrame({
    'A': np.random.normal(5.0, 0.1, 10),
    'B': np.random.normal(5.0, 0.1, 10),
    'C': np.random.normal(5.0, 0.1, 10),
    'D': np.random.normal(5.0, 0.1, 10),
    'Target': np.random.normal(50.0, 0.5, 10)
})

data_1_2 = pd.concat([cluster1, cluster2], ignore_index=True)

# Introduzir 10% missings (2 valores) - um de cada cluster
missing_indices_1_2 = [(2, 'Target'), (15, 'Target')]  # Linha 2 cluster1, linha 15 cluster2

result, imputer, mae = run_test_with_diagnostics(
    data_1_2,
    missing_indices=missing_indices_1_2,
    target_col='Target',
    test_name="1.2 - Dataset médio (20x5, 10% missing)"
)

# Verificação
val_cluster1 = result.loc[2, 'Target']
val_cluster2 = result.loc[15, 'Target']

assert 8 <= val_cluster1 <= 12, f"FALHOU cluster1: {val_cluster1} deveria estar em [8, 12]"
assert 48 <= val_cluster2 <= 52, f"FALHOU cluster2: {val_cluster2} deveria estar em [48, 52]"
print(f"\n✓ TESTE 1.2 PASSOU: cluster1={val_cluster1:.2f}, cluster2={val_cluster2:.2f}")

### Teste 1.3: Dataset maior (100 linhas, 10 features, 20% missing)

In [ ]:
np.random.seed(42)

# 3 clusters
n_per_cluster = 33

def make_cluster(center, target_center, n, n_features=10):
    data = {f'F{i}': np.random.normal(center, 0.2, n) for i in range(n_features)}
    data['Target'] = np.random.normal(target_center, 1.0, n)
    return pd.DataFrame(data)

cluster1 = make_cluster(1.0, 10.0, n_per_cluster)
cluster2 = make_cluster(5.0, 50.0, n_per_cluster)
cluster3 = make_cluster(9.0, 90.0, n_per_cluster + 1)  # +1 para dar 100

data_1_3 = pd.concat([cluster1, cluster2, cluster3], ignore_index=True)

# 20% missings aleatórios no Target
np.random.seed(42)
missing_rows = np.random.choice(100, 20, replace=False)
missing_indices_1_3 = [(row, 'Target') for row in missing_rows]

result, imputer, mae = run_test_with_diagnostics(
    data_1_3,
    missing_indices=missing_indices_1_3,
    target_col='Target',
    test_name="1.3 - Dataset maior (100x11, 20% missing)"
)

print(f"\nMAE para 20% missing: {mae:.4f}")
assert mae < 5.0, f"FALHOU: MAE {mae} muito alto"
print(f"✓ TESTE 1.3 PASSOU: MAE={mae:.4f} < 5.0")

### Teste 1.4: Dataset maior com 40% missing (verificar colapso)

In [ ]:
# Mesmo dataset, mas com 40% missings
np.random.seed(42)
missing_rows_40 = np.random.choice(100, 40, replace=False)
missing_indices_1_4 = [(row, 'Target') for row in missing_rows_40]

result, imputer, mae = run_test_with_diagnostics(
    data_1_3,  # Reutilizar dataset
    missing_indices=missing_indices_1_4,
    target_col='Target',
    test_name="1.4 - Dataset maior (100x11, 40% missing)"
)

print(f"\nMAE para 40% missing: {mae:.4f}")
print(f"Comparação: MAE 20%={mae:.4f} vs MAE 40%={mae:.4f}")

# Verificar se a Fase 2 foi activada
if imputer.execution_stats.get('phase2_activated', False):
    print(f"\n⚠️  FASE 2 ACTIVADA com {imputer.execution_stats['phase2_cycles']} ciclos")
else:
    print(f"\n✓ Fase 1 resolveu tudo")

---
## Teste 2: Correlação Forte vs Fraca

**Objectivo**: Verificar se os pesos MI reflectem correctamente as correlações reais.

Criamos um dataset onde:
- Coluna A tem correlação forte (0.95) com Target
- Coluna B tem correlação fraca (~0) com Target  
- Coluna C tem correlação média (0.5) com Target

In [ ]:
np.random.seed(42)
n = 100

# Target base
target = np.linspace(0, 100, n)

# A: correlação forte com Target (r ~ 0.95)
A = target + np.random.normal(0, 5, n)

# B: correlação fraca com Target (r ~ 0)
B = np.random.uniform(0, 100, n)

# C: correlação média com Target (r ~ 0.5)
C = 0.5 * target + 0.5 * np.random.uniform(0, 100, n)

data_2 = pd.DataFrame({
    'A': A,
    'B': B,
    'C': C,
    'Target': target
})

# Verificar correlações reais
from scipy.stats import pearsonr
corr_A = pearsonr(data_2['A'], data_2['Target'])[0]
corr_B = pearsonr(data_2['B'], data_2['Target'])[0]
corr_C = pearsonr(data_2['C'], data_2['Target'])[0]

print("=" * 70)
print("TESTE 2: Correlação Forte vs Fraca")
print("=" * 70)
print(f"\nCorrelações reais com Target:")
print(f"  A: {corr_A:.4f} (esperado ~0.95)")
print(f"  B: {corr_B:.4f} (esperado ~0.00)")
print(f"  C: {corr_C:.4f} (esperado ~0.50)")

In [ ]:
# Introduzir alguns missings e executar
np.random.seed(42)
missing_rows = np.random.choice(100, 20, replace=False)
missing_indices_2 = [(row, 'Target') for row in missing_rows]

result, imputer, mae = run_test_with_diagnostics(
    data_2,
    missing_indices=missing_indices_2,
    target_col='Target',
    test_name="2 - Correlação Forte vs Fraca"
)

# Verificar pesos MI
print(f"\n--- VERIFICAÇÃO DOS PESOS MI ---")
mi_A = imputer.mi_matrix.loc['A', 'Target']
mi_B = imputer.mi_matrix.loc['B', 'Target']
mi_C = imputer.mi_matrix.loc['C', 'Target']

print(f"MI(A, Target) = {mi_A:.4f}")
print(f"MI(B, Target) = {mi_B:.4f}")
print(f"MI(C, Target) = {mi_C:.4f}")

# Pesos normalizados
total_mi = mi_A + mi_B + mi_C
w_A = mi_A / total_mi
w_B = mi_B / total_mi
w_C = mi_C / total_mi

print(f"\nPesos normalizados:")
print(f"  w(A) = {w_A:.4f} (esperado > 0.5)")
print(f"  w(B) = {w_B:.4f} (esperado < 0.1)")
print(f"  w(C) = {w_C:.4f} (esperado ~0.3)")

# Verificações
assert w_A > w_C > w_B, f"FALHOU: ordem dos pesos deveria ser A > C > B, mas foi A={w_A:.4f}, B={w_B:.4f}, C={w_C:.4f}"
print(f"\n✓ TESTE 2 PASSOU: pesos MI reflectem correlações (A > C > B)")

---
## Teste 3: Fase 2 Forçada

**Objectivo**: Criar uma situação onde a Fase 1 não consegue imputar e verificar se a Fase 2 é activada correctamente.

In [ ]:
# Dataset onde cada linha tem missings em colunas diferentes
# Apenas uma linha está completa
data_3 = pd.DataFrame({
    'A': [1.0, np.nan, np.nan, np.nan, 1.0, 1.1, 1.2],
    'B': [np.nan, 2.0, np.nan, np.nan, 2.0, 2.1, 2.2],
    'C': [np.nan, np.nan, 3.0, np.nan, 3.0, 3.1, 3.2],
    'D': [np.nan, np.nan, np.nan, 4.0, 4.0, 4.1, 4.2],
    'Target': [10.0, 20.0, 30.0, 40.0, 25.0, 26.0, 27.0]  # Todos preenchidos
})

print("=" * 70)
print("TESTE 3: Fase 2 Forçada")
print("=" * 70)
print(f"\nDataset com overlap mínimo entre linhas:")
print(data_3.to_string())
print(f"\nLinhas 0-3: cada uma só tem 1 feature preenchida (overlap mínimo)")
print(f"Linhas 4-6: completas (podem ser donors)")

In [ ]:
# Tentar imputar a linha 0 que só tem A preenchido
data_3_test = data_3.copy()
true_value = data_3_test.loc[0, 'B']
data_3_test.loc[0, 'B'] = np.nan  # Adicionar mais um missing

print(f"\nAdicionando missing na linha 0, coluna B")
print(f"Valor real: {true_value}")

imputer = ISCAkCore(verbose=True, use_pds=True, min_friends=2)
result = imputer.impute(data_3_test, interactive=False)

print(f"\n--- ESTATÍSTICAS DE EXECUÇÃO ---")
stats = imputer.execution_stats
print(f"  Fase 2 activada: {stats.get('phase2_activated', 'N/A')}")
print(f"  Fase 2 ciclos: {stats.get('phase2_cycles', 'N/A')}")
print(f"  Fase 2 imputados: {stats.get('phase2_imputed', 'N/A')}")

print(f"\nResultado:")
print(result.to_string())

---
## Teste 4: PDS vs Clássico

**Objectivo**: Comparar o comportamento com e sem PDS (Partial Distance Strategy).

In [ ]:
# Dataset onde alguns donors têm overlap parcial
data_4 = pd.DataFrame({
    'A': [1.0, np.nan, 1.0, 5.0, 1.1],
    'B': [1.0, np.nan, 1.0, 5.0, 1.1],
    'C': [1.0, 1.0, 1.0, 5.0, 1.1],
    'D': [np.nan, 1.0, 1.0, 5.0, 1.1],
    'E': [np.nan, 1.0, 1.0, 5.0, np.nan],  # Linha 4 terá missing aqui
    'Target': [10.0, 10.0, 10.0, 50.0, np.nan]  # Linha 4 terá missing
})

print("=" * 70)
print("TESTE 4: PDS vs Clássico")
print("=" * 70)
print(f"\nDataset:")
print(data_4.to_string())
print(f"\nLinha 0: overlap 3/5 com linha 4 (A, B, C)")
print(f"Linha 1: overlap 2/5 com linha 4 (C, D)")
print(f"Linha 2: overlap 4/5 com linha 4 (A, B, C, D) - MELHOR")
print(f"Linha 3: overlap 4/5 com linha 4, mas distante")

In [ ]:
# Guardar valor real
true_target = 10.5  # Deveria ser ~10 baseado no cluster

# Teste COM PDS
print(f"\n{'='*30} COM PDS {'='*30}")
imputer_pds = ISCAkCore(verbose=True, use_pds=True, min_friends=2)
result_pds = imputer_pds.impute(data_4.copy(), interactive=False)
val_pds = result_pds.loc[4, 'Target']
print(f"\nValor imputado COM PDS: {val_pds:.4f}")

# Teste SEM PDS
print(f"\n{'='*30} SEM PDS {'='*30}")
imputer_no_pds = ISCAkCore(verbose=True, use_pds=False, min_friends=2)
result_no_pds = imputer_no_pds.impute(data_4.copy(), interactive=False)
val_no_pds = result_no_pds.loc[4, 'Target']
print(f"\nValor imputado SEM PDS: {val_no_pds:.4f}")

print(f"\n--- COMPARAÇÃO ---")
print(f"  COM PDS: {val_pds:.4f}")
print(f"  SEM PDS: {val_no_pds:.4f}")
print(f"  Esperado: ~10.0")

---
## Teste 5: Dados Mistos (Numérico + Categórico)

**Objectivo**: Verificar se a combinação de distâncias numéricas e categóricas está correcta.

In [ ]:
# Dataset misto com clusters baseados em numérico E categórico
data_5 = pd.DataFrame({
    'Num1': [1.0, 1.1, 1.2, 5.0, 5.1, 5.2, 1.05],
    'Num2': [10.0, 10.1, 10.2, 50.0, 50.1, 50.2, 10.05],
    'Cat1': ['A', 'A', 'A', 'B', 'B', 'B', 'A'],
    'Cat2': ['X', 'X', 'X', 'Y', 'Y', 'Y', 'X'],
    'Target': [100.0, 101.0, 102.0, 500.0, 501.0, 502.0, 100.5]  # Linha 6 será missing
})

print("=" * 70)
print("TESTE 5: Dados Mistos")
print("=" * 70)
print(f"\nDataset com numéricos e categóricos:")
print(data_5.to_string())
print(f"\nCluster 1 (linhas 0-2, 6): Num~1, Cat=A/X, Target~100")
print(f"Cluster 2 (linhas 3-5): Num~5, Cat=B/Y, Target~500")

In [ ]:
# Teste 5.1: Missing no Target (numérico)
print(f"\n{'='*30} TESTE 5.1: Missing numérico {'='*30}")
result, imputer, mae = run_test_with_diagnostics(
    data_5,
    missing_indices=[(6, 'Target')],
    target_col='Target',
    test_name="5.1 - Dados Mistos (missing numérico)"
)

val = result.loc[6, 'Target']
assert 90 <= val <= 110, f"FALHOU: {val} deveria estar em [90, 110]"
print(f"\n✓ TESTE 5.1 PASSOU: valor imputado {val:.2f} está no cluster correcto")

In [ ]:
# Teste 5.2: Missing no Cat1 (categórico)
print(f"\n{'='*30} TESTE 5.2: Missing categórico {'='*30}")

data_5_cat = data_5.copy()
true_cat = data_5_cat.loc[6, 'Cat1']
data_5_cat.loc[6, 'Cat1'] = np.nan

imputer = ISCAkCore(verbose=True, use_pds=True, min_friends=2)
result = imputer.impute(data_5_cat, interactive=False)

val_cat = result.loc[6, 'Cat1']
print(f"\nValor real: {true_cat}")
print(f"Valor imputado: {val_cat}")

# Nota: o valor pode estar codificado, precisamos decodificar
print(f"\nMapeamento nominal: {imputer.mixed_handler.nominal_mappings.get('Cat1', 'N/A')}")

---
## Teste 6: Simulação de Colapso (como SONAR 40%)

**Objectivo**: Tentar reproduzir o colapso que vimos no SONAR a 40% missing.

In [ ]:
# Criar dataset similar ao SONAR (muitas features)
np.random.seed(42)
n_samples = 200
n_features = 60

# 3 clusters
def make_sonar_like_cluster(center, target_center, n):
    data = {f'F{i}': np.random.normal(center, 0.2, n) for i in range(n_features)}
    data['Target'] = np.random.normal(target_center, 2.0, n)
    return pd.DataFrame(data)

c1 = make_sonar_like_cluster(0.2, 10, 66)
c2 = make_sonar_like_cluster(0.5, 50, 67)
c3 = make_sonar_like_cluster(0.8, 90, 67)

data_6 = pd.concat([c1, c2, c3], ignore_index=True)

print("=" * 70)
print("TESTE 6: Simulação de Colapso (tipo SONAR)")
print("=" * 70)
print(f"\nDataset: {data_6.shape[0]} linhas x {data_6.shape[1]} colunas")
print(f"3 clusters com targets ~10, ~50, ~90")

In [ ]:
# Teste com 20% missing
print(f"\n{'='*30} 20% MISSING {'='*30}")
np.random.seed(42)
missing_20 = np.random.choice(200, 40, replace=False)
missing_indices_20 = [(row, 'Target') for row in missing_20]

data_6_20 = data_6.copy()
true_vals_20 = {row: data_6.loc[row, 'Target'] for row in missing_20}
for row in missing_20:
    data_6_20.loc[row, 'Target'] = np.nan

imputer_20 = ISCAkCore(verbose=True, use_pds=True)
result_20 = imputer_20.impute(data_6_20, interactive=False)

# Calcular MAE
errors_20 = [abs(result_20.loc[row, 'Target'] - true_vals_20[row]) for row in missing_20]
mae_20 = np.mean(errors_20)
print(f"\nMAE 20%: {mae_20:.4f}")
print(f"Fase 2 activada: {imputer_20.execution_stats.get('phase2_activated', 'N/A')}")

In [ ]:
# Teste com 40% missing
print(f"\n{'='*30} 40% MISSING {'='*30}")
np.random.seed(42)
missing_40 = np.random.choice(200, 80, replace=False)
missing_indices_40 = [(row, 'Target') for row in missing_40]

data_6_40 = data_6.copy()
true_vals_40 = {row: data_6.loc[row, 'Target'] for row in missing_40}
for row in missing_40:
    data_6_40.loc[row, 'Target'] = np.nan

imputer_40 = ISCAkCore(verbose=True, use_pds=True)
result_40 = imputer_40.impute(data_6_40, interactive=False)

# Calcular MAE
errors_40 = [abs(result_40.loc[row, 'Target'] - true_vals_40[row]) for row in missing_40]
mae_40 = np.mean(errors_40)
print(f"\nMAE 40%: {mae_40:.4f}")
print(f"Fase 2 activada: {imputer_40.execution_stats.get('phase2_activated', 'N/A')}")

In [ ]:
# Comparação
print(f"\n{'='*30} COMPARAÇÃO {'='*30}")
print(f"MAE 20% missing: {mae_20:.4f}")
print(f"MAE 40% missing: {mae_40:.4f}")
print(f"Degradação: {((mae_40 - mae_20) / mae_20 * 100):.1f}%")

if mae_40 > mae_20 * 3:
    print(f"\n⚠️  COLAPSO DETECTADO: MAE triplicou de 20% para 40%")
else:
    print(f"\n✓ Degradação dentro do esperado")

---
## Teste 7: Adaptive K

**Objectivo**: Verificar se o adaptive k escolhe valores apropriados em diferentes cenários.

In [ ]:
print("=" * 70)
print("TESTE 7: Adaptive K")
print("=" * 70)

# Cenário 7.1: Vizinhos CONSISTENTES (todos com target similar)
print(f"\n--- Cenário 7.1: Vizinhos Consistentes ---")
distances_consistent = np.array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0])
values_consistent = np.array([10.0, 10.1, 10.2, 9.9, 10.0, 10.1, 9.8, 10.2, 10.0, 10.1])

k_consistent = adaptive_k_hybrid(distances_consistent, values_consistent, min_k=3, max_k=10, alpha=0.5)
print(f"  Targets dos vizinhos: {values_consistent}")
print(f"  Std dos targets: {np.std(values_consistent):.4f}")
print(f"  k escolhido: {k_consistent}")
print(f"  Esperado: k ALTO (vizinhos concordam, seguro usar mais)")

# Cenário 7.2: Vizinhos INCONSISTENTES (targets muito diferentes)
print(f"\n--- Cenário 7.2: Vizinhos Inconsistentes ---")
distances_inconsistent = np.array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0])
values_inconsistent = np.array([10.0, 50.0, 5.0, 100.0, 20.0, 80.0, 15.0, 90.0, 30.0, 70.0])

k_inconsistent = adaptive_k_hybrid(distances_inconsistent, values_inconsistent, min_k=3, max_k=10, alpha=0.5)
print(f"  Targets dos vizinhos: {values_inconsistent}")
print(f"  Std dos targets: {np.std(values_inconsistent):.4f}")
print(f"  k escolhido: {k_inconsistent}")
print(f"  Esperado: k BAIXO (vizinhos discordam, usar menos)")

# Verificação
assert k_consistent > k_inconsistent, f"FALHOU: k_consistent ({k_consistent}) deveria ser > k_inconsistent ({k_inconsistent})"
print(f"\n✓ TESTE 7 PASSOU: k_consistent ({k_consistent}) > k_inconsistent ({k_inconsistent})")

---
## Resumo dos Testes

Execute esta célula no final para ver um resumo de todos os testes.

In [ ]:
print("\n" + "=" * 70)
print("RESUMO DOS TESTES DIAGNÓSTICOS")
print("=" * 70)
print("""
Execute cada teste individualmente e verifique:

1. VIZINHOS ÓBVIOS
   - Os vizinhos seleccionados são do cluster correcto?
   - O valor imputado está no range esperado?
   - Há degradação excessiva com mais missings?

2. CORRELAÇÃO FORTE vs FRACA
   - Os pesos MI reflectem as correlações reais?
   - A ordem é A > C > B?

3. FASE 2 FORÇADA
   - A Fase 2 é activada quando necessário?
   - Quantos ciclos são necessários?

4. PDS vs CLÁSSICO
   - O PDS melhora ou piora os resultados?
   - A escala está correcta?

5. DADOS MISTOS
   - Distâncias numéricas e categóricas estão calibradas?
   - A votação categórica funciona?

6. COLAPSO 40%
   - Há colapso súbito entre 20% e 40%?
   - A Fase 2 está a ser usada excessivamente?

7. ADAPTIVE K
   - k varia conforme esperado?
   - Vizinhos consistentes → k alto?
   - Vizinhos inconsistentes → k baixo?
""")